In [ ]:
import pandas as pd
import numpy as np
import re

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


- 우대조건 전처리
- 구분 전처리
- 은행안정성 합치기

- 컬럼 동일하게 맞추기

In [ ]:
parking = pd.read_csv("/content/drive/MyDrive/ASAC/final/data/상품데이터/파킹/final_parking.csv")

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/ASAC/final/data/상품데이터/최종합본/예금/최종본/final_deposit_all.csv", encoding = 'cp949')

In [ ]:
parking.columns

Index(['금융회사', '상품명', '구간', '세전이자율', '세후이자율', '세후이자(예시)', '최고우대금리', '실질이자율',
       '비교 공시일', '담당부서 및 연락처', '상품유형', '우대조건', '가입대상', '가입방법', '금액', '이자지급',
       '기타유의사항', '예금자\n보호', '유형', '등급', '우대사항정리', '급여이체', '마케팅동의', '자동이체',
       '카드실적', '최초거래', '주택청약', '비대면', '영업점', '해당은행', '이체횟수', '만기', '변동금리',
       '이벤트', '기타', '나이', '환경', '신혼', '반려인', '직장인', '여행', '금액구간'],
      dtype='object')

In [ ]:
df.columns

Index(['금융회사', '상품명', '적립방식', '세전이자율', '세후이자율', '세후이자(예시)', '최고우대금리', '실질이자율',
       '이자계산방식', '기간', '비교 공시일', '담당부서 및 연락처', '우대조건', '가입대상.1', '가입방법',
       '만기후 이자율', '기타유의사항', '등급', 'cluster_num', '우대사항정리', '급여이체', '마케팅동의',
       '자동이체', '카드실적', '최초거래', '주택청약', '비대면', '영업점', '해당은행', '이체횟수', '만기',
       '변동금리', '이벤트', '기타', '나이', '환경', '신혼', '반려인', '직장인', '여행'],
      dtype='object')

In [ ]:
parking['적립방식'] = np.nan
parking['이자계산방식'] = np.nan
parking['만기후 이자율'] = np.nan
parking['cluster_num'] = np.nan
parking.rename(columns ={'구간': '기간', '가입대상':'가입대상.1'}, inplace = True)
parking.drop(columns = "금액", inplace = True)
parking.drop(columns = "유형", inplace = True)
parking.drop(columns = "상품유형", inplace = True)
parking.drop(columns = "예금자\n보호", inplace = True)

In [ ]:
parking.drop(columns = "금액구간", inplace = True)

In [ ]:
# 파킹에만 있는 것 => 이자지급

In [ ]:
parking = parking[['금융회사', '상품명', '적립방식', '세전이자율', '세후이자율', '세후이자(예시)', '최고우대금리', '실질이자율',
       '이자계산방식', '기간', '이자지급', '비교 공시일', '담당부서 및 연락처', '우대조건', '가입대상.1', '가입방법',
       '만기후 이자율', '기타유의사항', '등급', 'cluster_num', '우대사항정리', '급여이체', '마케팅동의',
       '자동이체', '카드실적', '최초거래', '주택청약', '비대면', '영업점', '해당은행', '이체횟수', '만기',
       '변동금리', '이벤트', '기타', '나이', '환경', '신혼', '반려인', '직장인', '여행']]

In [ ]:
parking.to_csv("/content/drive/MyDrive/ASAC/final/data/상품데이터/파킹/final_parking_new_columns.csv")

- 기간별 삭제

In [ ]:
parking = pd.read_csv("/content/drive/MyDrive/ASAC/final/data/상품데이터/파킹/final_parking_new_columns.csv")

In [ ]:
parking['기간별'] = parking['상품명'].apply(lambda x : 1 if "기간" in x else 0)
parking = parking[parking['기간별'] == 0]
parking.drop(columns = '기간별', inplace = True)

<ipython-input-87-bb6e28613caa>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  parking.drop(columns = '기간별', inplace = True)


- 구간 (기간 ) 전처리

In [ ]:
parking.reset_index(drop = True, inplace = True)

In [ ]:
parking['기간'] = parking['기간'].astype("str")

In [ ]:
parking['기간'] = parking['기간'].fillna("제한없음")

In [ ]:
parking['기간']  = parking['기간'].apply(lambda x : x.replace("억", "0000만") if "억" in x else x)
# for i in range(len(parking)):
#     parking.loc[i, '기간'] = parking.loc[i, '기간'].replace(",","")

In [ ]:
parking['기간'] = parking['기간'].apply(lambda x : x.replace(" ",""))

In [ ]:
parking['기간'].unique()

array(['50만원이하', '50만원초과', '100만원이하', '100만원초과~500만원이하', '500만원초과',
       '200만원이하', '2000만원이하', '500만원이하', '30000만원미만', '30000만원이상',
       '5000만원이하', '5000만원초과~10000만원이하', '10000만원초과', '제한없음', '300만원이하',
       '300만원초과', '5000만원초과', '1000만원이하', '1000만원초과~10000만원이하',
       '10000만원이하', '10000만원초과~100000만원이하', '100000만원초과', '30만원미만',
       '30만원이상~100만원미만', '100만원이상~200만원미만', '200만원이상', '30000만원이하',
       '30000만원초과', '1일이상', '50만원초과~500만원이하', '1000000만원이하',
       '300만원초과~20000만원이하', '20000만원초과', '100000만원미만',
       '100000만원이상~300000만원미만', '300000만원이상~500000만원미만', '500000만원이상',
       '50만원초과~200만원이하', '200만원초과', '100만원초과', '500000만이하', '500000만초과',
       '20000만원이하', '500만원미만', '500만원이상', '10000만원미만', '50000만원미만',
       '50000만원이상', '100000만원이하', '1000만원초과', '5000만원미만', '5000만원이상',
       '2000만원초과', '10만원미만', '100만원미만', '1000만원미만', '1000만원이상',
       '3000만원이상', '3000만원미만', '300만원미만', '500만원초과~1000만원이하', '10000만원이상',
       '1000000만이하', '1000000만초과', '150만원이하', '150만원초과~

In [ ]:
# for i in range(len(parking)):
#     if '초과' in parking.loc[i, '기간']  and '이하' in parking.loc[i, '기간'] :
#         parking.loc[i, '기간'] = " 초과 ~ ".join(parking.loc[i, '기간'].split('초과'))
#     elif '이상' in parking.loc[i, '기간']  and '미만' in parking.loc[i, '기간'] :
#         parking.loc[i, '기간'] = " 이상 ~ ".join(parking.loc[i, '기간'].split('이상'))
#     elif '초과' in parking.loc[i, '기간']  and '미만' in parking.loc[i, '기간'] :
#         parking.loc[i, '기간'] = " 초과 ~ ".join(parking.loc[i, '기간'].split('초과'))

In [ ]:
# 1일 이상 지우기

In [ ]:
parking.to_csv("/content/drive/MyDrive/ASAC/final/data/상품데이터/파킹/final_parking_new_columns.csv")

In [ ]:
parking = pd.read_csv("/content/drive/MyDrive/ASAC/final/data/상품데이터/파킹/final_parking_new_columns.csv")

In [ ]:
p = re.compile("[0-9]+")
p.findall('10억원이상~억원미만')

['10']

In [ ]:
p = re.compile("[0-9]+")
def parking_m_cal(m_range , money):

    if "~" in m_range:
        if "초과" in m_range and "이하" in m_range:
            m_range_list = p.findall(m_range)
            if int(m_range_list[0] )< money and money <= int(m_range_list[1]):
                return 1

        if "초과" in m_range and "미만" in m_range:
            m_range_list = p.findall(m_range)
            if int(m_range_list[0] )< money and money < int(m_range_list[1]):
                return 1

        if "이상" in m_range and "미만" in m_range:
            m_range_list = p.findall(m_range)
            if (int(m_range_list[0] )<= money) and  (money < int(m_range_list[1])):
                    return 1
    else:
        if "이하" in m_range:
            if  int(p.findall(m_range)[0]) >= money :
                return 1
        if "미만" in m_range:
            if  int(p.findall(m_range)[0]) > money :
                return 1
        if "이상" in m_range:
            if  int(p.findall(m_range)[0]) <= money :
                return 1
        if "초과" in m_range:
            if  int(p.findall(m_range)[0]) < money :
                return 1

    return 0


In [ ]:
parking['기간적합'] = parking['기간'].apply(lambda x : parking_m_cal(x,100))

In [ ]:
parking[parking['기간적합'] == 1][['기간','상품명']]

,기간,상품명
1,50만원초과,OK짠테크통장
3,50만원초과,OK페이통장
4,100만원이하,OK읏백만통장Ⅱ
7,200만원이하,머니모으기
8,2000만원이하,플러스자유예금
...,...,...
201,50만원이상,아낌없이 주는 저축예금
203,50만원이상,아낌없이 주는 저축예금
204,100만원이하,우리아이행복통장
206,500만원미만,저축예금


In [ ]:
def prefer(df_client, idx, df):
     # 우대사항 처리
    col_list_all = ['금융회사', '상품명', '적립방식', '세전이자율', '세후이자율', '세후이자(예시)', '최고우대금리','실질이자율',
        '이자계산방식', '기간', '비교 공시일', '담당부서 및 연락처', '우대조건', '가입대상.1', '가입방법', '만기후 이자율', '기타유의사항','등급', 'cluster_num','우대사항정리']

    # 선택한 우대사항 리스트만 선택해서 가지고오기
    list_prefer = ["만기"]

    for prefer in df_client.loc[idx, '우대사항'].split(","):
        list_prefer.append(prefer.strip())

    if "해당사항없음" in list_prefer:
        list_prefer.remove("해당사항없음")

    client_list = col_list_all + list_prefer
    # print(client_list)
    df = df[client_list]

        # 우대사항 값계산
    def rate_cal(x):
        if df_client.loc[idx, '수시입출금'] == "예":
            total_ = float(x['세전이자율'] * 100)
        else:
            try:
                total_ = float(x['세전이자율'][:-1])
            except:
                total_ = float(x['최고우대금리'][:-1])

        for c in list_prefer:
            if "%" in str(x[c]):
                # print(x[c])
                total_+= float(x[c].replace("%",""))
            else:
                try:
                    total_ += float(x[c])
                except:
                    pass

        return total_

    # print(df.apply(lambda x : rate_cal(x), axis = 1))
    df['최종금리'] = df.apply(lambda x : rate_cal(x), axis = 1)
    print(df['최종금리'])
    return df

In [ ]:
    def rate_cal(x):
        if df_client.loc[idx, '수시입출금'] == "예":
            total_ = float(x['세전이자율'] * 100)
        else:
            try:
                total_ = float(x['세전이자율'][:-1])
            except:
                total_ = float(x['최고우대금리'][:-1])

        for c in list_prefer:
            if "%" in str(x[c]):
                # print(x[c])
                total_+= float(x[c].replace("%",""))
            else:
                try:
                    total_ += float(x[c])
                except:
                    pass

        return total_

In [ ]:
parking['세전이자율'] * 100

0      7.00
1      3.50
2      4.00
3      0.50
4      4.50
       ... 
207    0.50
208    1.00
209    0.50
210    0.10
211    0.05
Name: 세전이자율, Length: 212, dtype: float64

In [ ]:
# total_ = float(x['세전이자율'] * 100)
def aa(x):
    list_prefer = ["만기", "비대면"]
    total_ = float(x['세전이자율'] * 100)
    for c in list_prefer:
        print(x[c])
        # total_+= float(x[c].replace("%",""))
parking.apply(lambda x: aa(x), axis = 1)

nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
2.0%
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
0.1%
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
0.1%
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
0.1%
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan


0      None
1      None
2      None
3      None
4      None
       ... 
207    None
208    None
209    None
210    None
211    None
Length: 212, dtype: object

In [ ]:
parking['세전이자율'].info()

<class 'pandas.core.series.Series'>
RangeIndex: 212 entries, 0 to 211
Series name: 세전이자율
Non-Null Count  Dtype  
--------------  -----  
212 non-null    float64
dtypes: float64(1)
memory usage: 1.8 KB
